In [2]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import statsmodels.api as sm
from time import time
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from random import randint
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Dropout,Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras import initializers
from matplotlib import pyplot
from datetime import datetime
import sklearn.model_selection
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import keras
from sklearn.metrics import mean_squared_error
from math import sqrt
py.init_notebook_mode(connected=True)
%matplotlib inline


##**IMPORT DATASET**

In [0]:
# Read the dataset and record the running time
time_1 = time()
data_df = pd.read_csv('LSTM_Dataset.csv', sep = ',', parse_dates=[0], dayfirst = True)
time_2 = time()
print('read data cost '+ str(time_2 - time_1)+' second')

Saving LSTM_Dataset.csv to LSTM_Dataset.csv
User uploaded file "LSTM_Dataset.csv" with length 92132 bytes
read data cost 0.019783496856689453 second


In [0]:
# Split data into features and labels
n_samples = len(data_df)
X = data_df.iloc[0:n_samples-1,0:-1].values
Y = data_df.iloc[0:n_samples-1,-1].values

X_train = X[:-91]
X_test = X[-91:]
Y_train = Y[:-91]
Y_test = Y[-91:]


# Normalize data for MLP
scaler = MinMaxScaler(feature_range=(0, 1))
MLP_X_train = scaler.fit_transform(X_train)
MLP_X_test = scaler.transform(X_test)

scaler2 = MinMaxScaler(feature_range=(0, 1))
MLP_Y_train = scaler2.fit_transform(Y_train.reshape(-1,1))
MLP_Y_test = scaler2.transform(Y_test.reshape(-1,1))


# Normalize data for LSTM,GRU
scaler1 = MinMaxScaler()
scaler1.fit(X_train,Y_train)
X_train = scaler1.transform(X_train).reshape(len(X_train),X_train.shape[1],1)
X_test = scaler1.transform(X_test).reshape(len(X_test),X_test.shape[1],1)

scaler2 = MinMaxScaler()
scaler2.fit(Y_train.reshape(-1,1))
Y_train = scaler2.transform(Y_train.reshape(-1,1))
Y_test = scaler2.transform(Y_test.reshape(-1,1))

In [0]:
print(MLP_Y_train.shape)
print(Y_train.shape)

(819, 1)
(819, 1)


##**DEFINE FUNCTIONS**

In [0]:
### DISPLAY PLOTLY GRAPHS IN COLAB
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
# The root_mean_squared_error
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
    
### PREDICTION USING TEST DATA
def predict(model):
  global X_test, Y_test
  X_test = np.reshape(X_test, (len(X_test), 1, 1))
  # get predictions and then make some transformations to be able to calculate RMSE properly in USD
  prediction = model.predict(X_test) 
  prediction_inverse = scaler.inverse_transform(prediction.reshape(-1, 1)) 
  Y_test_inverse = scaler.inverse_transform(Y_test.reshape(-1, 1)) 
  prediction2_inverse = np.array(prediction_inverse[:,0][:]) 
  Y_test2_inverse = np.array(Y_test_inverse[:,0]) 
  RMSE = sqrt(mean_squared_error(Y_test2_inverse, prediction2_inverse))
  #print('Test RMSE: %.3f' % RMSE)
  return RMSE

## TRAIN MLP MODEL:
def MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, n_neurons, n_layers, batch_size, epochs, dropout_rate):
  model = Sequential()
  for i in range(0,n_layers):
    if i == 0: 
      if n_layers == 1: model.add(Dense(n_neurons, input_dim=len(X_train[0])))
      elif n_layers > 1: model.add(Dense(n_neurons, input_dim=len(X_train[0])))
    if i > 0:
      if i <= n_layers - 1: model.add(Dense(n_neurons))
  if dropout_rate > 0:
    model.add(Dropout(dropout_rate))
  model.add(Dense(1))
  time_3 = time()
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
  # compile and fit the model
  model.compile(loss=root_mean_squared_error, optimizer=optimizer)
  print(model.summary())
  history = model.fit(MLP_X_train, MLP_Y_train, epochs=epochs, batch_size=batch_size,
                      validation_data=(MLP_X_test, MLP_Y_test))
                      #callbacks = [EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=40, verbose=1)])
  time_4 = time()
  print('Training cost '+ str(time_4 - time_3)+' second')
  return history, model

### TRAIN LSTM MODEL:
def LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, n_neurons, n_layers, batch_size, epochs, dropout_rate):
  model = Sequential()
  for i in range(0,n_layers):
    if i == 0:
      if n_layers == 1: model.add(LSTM(n_neurons, input_shape=(X_train.shape[1], X_train.shape[2])))
      elif n_layers > 1: model.add(LSTM(n_neurons, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    if i > 0:
      if i < n_layers-1: model.add(LSTM(n_neurons, return_sequences=True))
      elif i == n_layers-1: model.add(LSTM(n_neurons))
  if dropout_rate > 0:
    model.add(Dropout(dropout_rate))
  model.add(Dense(1))

  time_5 = time()
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
  # compile and fit the model
  model.compile(loss=root_mean_squared_error, optimizer=optimizer)
  print(model.summary())
  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=False,
                      validation_data=(X_test, Y_test), verbose = 1)
                      #callbacks = [EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=40, verbose=1)])
  time_6 = time()
  print('Training cost '+ str(time_6 - time_5)+' second')
  return history, model

## TRAIN GRU MODEL:
def GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, n_neurons, n_layers, batch_size, epochs, dropout_rate):
  model = Sequential()
  for i in range(0,n_layers):
    if i == 0:
      if n_layers == 1: model.add(GRU(n_neurons, input_shape=(X_train.shape[1], X_train.shape[2])))
      elif n_layers > 1: model.add(GRU(n_neurons, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    if i > 0:
      if i < n_layers-1: model.add(GRU(n_neurons, return_sequences=True))
      elif i == n_layers-1: model.add(GRU(n_neurons))
  if dropout_rate > 0:
    model.add(Dropout(dropout_rate))
  model.add(Dense(1))

  time_7 = time()
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
  # compile and fit the model
  model.compile(loss=root_mean_squared_error, optimizer=optimizer)
  print(model.summary())
  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=False,
                      validation_data=(X_test, Y_test), verbose = 1)
                      #callbacks = [EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=40, verbose=1)])
  time_8 = time()
  print('Training cost '+ str(time_8 - time_7)+' second')
  return history, model

### PLOT LOSS GRAPHS
def plot_loss(loss, parameter, title, label):

  size = len(loss[0])
  colors = ['blue','red','green','orange','grey']
  data = []
  for i in range(0,len(loss)):
    trace = go.Scatter(
        x = np.arange(0, size, 1),
        y = loss[i],
        mode = 'lines',
        name = label + ' = ' + str(parameter[i]),
        line = dict(color=colors[i], width=2)
    )
    data.append(trace)
  layout = dict(title = 'Test Loss of different ' + title,
                xaxis = dict(title = 'Epoch number'), yaxis = dict(title = 'Loss'))
  fig = dict(data=data, layout=layout)
  py.iplot(fig, filename='training_process')
  
  
  
  # Get best learning rate, loss
  min_loss = []
  for i in range(0,len(loss)):
    min_loss.append(np.min(loss[i]))
    
  best_parameter = parameter[np.argmin(min_loss)]
  best_loss = np.min(min_loss)  

  return best_parameter, best_loss


## **MLP BENCHMARK**

In [0]:
# Define the model
model = keras.models.Sequential()
model.add(Dense(128,input_dim=len(X_train[0])))
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Dense(1))
adam = Adam(lr=0.0005, decay=1e-6)
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
model.compile(loss=root_mean_squared_error, optimizer=adam)
print(model.summary())

history = model.fit(MLP_X_train, MLP_Y_train,epochs=100, batch_size=16,
                    validation_data=(MLP_X_test, MLP_Y_test),
                   callbacks = [EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=20, verbose=1)])
y_pred = model.predict(MLP_X_test)
y_true = scaler2.inverse_transform(MLP_Y_test)
y_hat = scaler2.inverse_transform(y_pred)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_35 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 129       
Total params: 17,409
Trainable params: 17,409
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 1s 748us/step - loss: 0.1234 - val_loss: 0.0167
Epoch 2/100
819/819 [==============================] - 0s 331us/step - loss: 0.0649 - val_loss: 0.0537
Epoch 3/100
81

##**OPTIMIZER FINE-TUNING**

### MLP

#### Adam


In [0]:
## Adam get the best learning rate
time_1 = time()

loss = []
learning_rate = [1e-7,1e-6,1e-5,1e-4,1e-3]   
for lr in learning_rate:
  optimizer = Adam(lr=lr, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 128, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               768       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/200
819/819 [==============================] - 1s 1ms/step - loss: 0.2229 - val_loss: 0.4766
Epoch 2/200
819/819 [==============================] - 0s 288us/step - loss: 0.2227 - val_loss: 0.4763
Epoch 3/200
819/819 [==============================] - 0s 264us/step - loss: 0.2224 - val_loss: 0.4761
Epoch 4/200
819/819 [==============================] - 0s 262us/step - loss: 0.2221 - val_loss: 0.4758
Epoch 5/200
819/819 [==============================] - 0s 289us/step - loss: 0.2

In [0]:
configure_plotly_browser_state()

best_learning_rate_Adam, best_loss_Adam = plot_loss(loss,learning_rate, 'Adam learnig rates', 'lr')
print('Best learning rate:', str(best_learning_rate_Adam))
print('Best loss:', str(best_loss_Adam))

Best learning rate: 0.001
Best loss: 0.01253836364536495


#### SGD


In [0]:
## SGD get the best learning rate
time_1 = time()

loss = []
learning_rate = [1e-7,1e-6,1e-5,1e-4,1e-3]   
for lr in learning_rate:
  optimizer = SGD(lr=lr, momentum=0.9, decay=1e-6, nesterov=True)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 128, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 0s 503us/step - loss: 0.1689 - val_loss: 0.3785
Epoch 2/100
819/819 [==============================] - 0s 241us/step - loss: 0.1687 - val_loss: 0.3782
Epoch 3/100
819/819 [==============================] - 0s 227us/step - loss: 0.1685 - val_loss: 0.3780
Epoch 4/100
819/819 [==============================] - 0s 230us/step - loss: 0.1683 - val_loss: 0.3777
Epoch 5/100
819/819 [==============================] - 0s 225us/step - loss: 0

In [0]:
configure_plotly_browser_state()

best_learning_rate_SGD, best_loss_SGD = plot_loss(loss,learning_rate, 'SGD learnig rates', 'lr')
print('Best learning rate:', str(best_learning_rate_SGD))
print('Best loss:', str(best_loss_SGD))

Best learning rate: 0.001
Best loss: 0.012741404575305981


#### RMSprop

In [0]:
## RMSprop get the best learning rate
time_1 = time()

loss = []
learning_rate = [1e-7,1e-6,1e-5,1e-4,1e-3]   
for lr in learning_rate:
  optimizer = optimizer = RMSprop(lr=lr, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 128, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 0s 606us/step - loss: 0.1013 - val_loss: 0.2152
Epoch 2/100
819/819 [==============================] - 0s 263us/step - loss: 0.1012 - val_loss: 0.2150
Epoch 3/100
819/819 [==============================] - 0s 270us/step - loss: 0.1012 - val_loss: 0.2148
Epoch 4/100
819/819 [==============================] - 0s 254us/step - loss: 0.1011 - val_loss: 0.2146
Epoch 5/100
819/819 [==============================] - 0s 257us/step - loss: 0

In [0]:
configure_plotly_browser_state()

best_learning_rate_RMSprop, best_loss_RMSprop = plot_loss(loss,learning_rate, 'RMSprop learnig rates', 'lr')
print('Best learning rate:', str(best_learning_rate_RMSprop))
print('Best loss:', str(best_loss_RMSprop))

Best learning rate: 0.001
Best loss: 0.012361956166697073


#### RMSprop Tuning-parameters


In [0]:
# RMSprop Get the best number of neurons
time_1 = time()

loss = []
n_neurons_options = [128,256,512]   
for n_neurons in n_neurons_options:
  optimizer =RMSprop(lr=0.001, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, n_neurons, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 1s 1ms/step - loss: 0.0580 - val_loss: 0.0353
Epoch 2/100
819/819 [==============================] - 0s 259us/step - loss: 0.0348 - val_loss: 0.0145
Epoch 3/100
819/819 [==============================] - 0s 291us/step - loss: 0.0342 - val_loss: 0.0270
Epoch 4/100
819/819 [==============================] - 0s 263us/step - loss: 0.0310 - val_loss: 0.0208
Epoch 5/100
819/819 [==============================] - 0s 260us/step - loss: 0.0

In [0]:
configure_plotly_browser_state()

best_number_of_neurons_RMSprop, best_loss_RMSprop = plot_loss(loss,n_neurons_options, 'RMSprop number of neurons', 'n_neurons')
print('Best_number_of_neurons_RMSprop:', str(best_number_of_neurons_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_number_of_neurons_RMSprop: 256
Best_loss_RMSprop: 0.012116796009570033


In [0]:
# Get the best number of layers
time_1 = time()

loss = []
n_layers_options = [1,2,3]   
for n_layers in n_layers_options:
  optimizer =RMSprop(lr=0.001, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 256, n_layers, 16, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 256)               1536      
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 257       
Total params: 1,793
Trainable params: 1,793
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 1s 1ms/step - loss: 0.0629 - val_loss: 0.0336
Epoch 2/100
819/819 [==============================] - 0s 262us/step - loss: 0.0426 - val_loss: 0.0749
Epoch 3/100
819/819 [==============================] - 0s 267us/step - loss: 0.0394 - val_loss: 0.0343
Epoch 4/100
819/819 [==============================] - 0s 255us/step - loss: 0.0394 - val_loss: 0.0175
Epoch 5/100
819/819 [==============================] - 0s 259us/step - loss:

In [0]:
configure_plotly_browser_state()

best_number_of_layers_RMSprop, best_loss_RMSprop = plot_loss(loss,n_layers_options, 'RMSprop number of layers', 'n_layers')
print('Best_number_of_layers_RMSprop:', str(best_number_of_layers_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_number_of_layers_RMSprop: 2
Best_loss_RMSprop: 0.012129047435227332


In [0]:
# Get the best number of batch_size
time_1 = time()

loss = []
n_batch_size = [16,1,32,64]   
for batch_size in n_batch_size:
  optimizer =RMSprop(lr=0.001, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 256, 2, batch_size, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 256)               1536      
_________________________________________________________________
dense_72 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 257       
Total params: 67,585
Trainable params: 67,585
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 1s 1ms/step - loss: 0.2135 - val_loss: 0.2332
Epoch 2/100
819/819 [==============================] - 0s 285us/step - loss: 0.1001 - val_loss: 0.0476
Epoch 3/100
819/819 [==============================] - 0s 277us/step - loss: 0.0740 - val_loss: 0.0840
Epoch 4/100
819/819 [========================

In [0]:
configure_plotly_browser_state()

best_number_of_batch_size_RMSprop, best_loss_RMSprop = plot_loss(loss,n_batch_size, 'RMSprop number of batch_size', 'batch_size')
print('Best_number_of_batch_size_RMSprop:', str(best_number_of_batch_size_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_number_of_batch_size_RMSprop: 64
Best_loss_RMSprop: 0.012014977449735443


In [0]:
# Get the best rate of dropout
time_1 = time()

loss = []
n_drop_rates = [0,0.25,0.5,0.75]   
for drop_rate in n_drop_rates:
  optimizer =RMSprop(lr=0.001, decay=1e-6)
  history, model = MLP_train_model(MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test, optimizer, 256, 2, 64, 100, drop_rate)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_83 (Dense)             (None, 256)               1536      
_________________________________________________________________
dense_84 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_85 (Dense)             (None, 1)                 257       
Total params: 67,585
Trainable params: 67,585
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 1s 1ms/step - loss: 0.3566 - val_loss: 0.2361
Epoch 2/100
819/819 [==============================] - 0s 90us/step - loss: 0.1776 - val_loss: 0.0924
Epoch 3/100
819/819 [==============================] - 0s 85us/step - loss: 0.1474 - val_loss: 0.0839
Epoch 4/100
819/819 [==========================

In [0]:
configure_plotly_browser_state()

best_dropout_rate_RMSprop, best_loss_RMSprop = plot_loss(loss,n_drop_rates, 'RMSprop number of dropout_rate', 'dropout_rate')
print('Best_dropout_rate_RMSprop:', str(best_dropout_rate_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_dropout_rate_RMSprop: 0.25
Best_loss_RMSprop: 0.012274120903604633


### LSTM

#### Adam

In [0]:
time_1 = time()

# Adam
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]   
loss_adam = []
for lr in learning_rate:
  optimizer = Adam(lr=lr, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_adam.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_95 (Dense)             (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 2s 3ms/step - loss: 0.1636 - val_loss: 0.4025
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1474 - val_loss: 0.3876
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1402 - val_loss: 0.3795
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1394 - val_loss: 0.3756
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.139

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_adam, best_loss_adam = plot_loss(loss_adam, learning_rate, 'Adam learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_adam))
print('Best loss:', str(best_loss_adam))



Best learning rate: 0.0001
Best loss: 0.013352941332773849


#### SGD

In [0]:
time_1 = time()

# RMSprop
learning_rate = [1e-4,5e-4,1e-3,5e-3,1e-2]    # MIGHT NEED TO ADJUST
loss_sgd = []
for lr in learning_rate:
  optimizer = SGD(lr=lr, momentum=0.9, decay=1e-6, nesterov=True)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_sgd.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_100 (Dense)            (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 3s 4ms/step - loss: 0.1214 - val_loss: 0.3096
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1435 - val_loss: 0.3208
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1382 - val_loss: 0.3202
Epoch 4/100
819/819 [==============================] - 1s 1000us/step - loss: 0.1379 - val_loss: 0.3189
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_sgd, best_loss_sgd = plot_loss(loss_sgd, learning_rate, 'SGD learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_sgd))
print('Best loss:', str(best_loss_sgd))


Best learning rate: 0.001
Best loss: 0.01620336056042176


#### RMSprop

In [0]:
time_1 = time()

# RMSprop
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]    # MIGHT NEED TO ADJUST
loss_rmsprop = []
for lr in learning_rate:
  optimizer = RMSprop(lr=lr, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_rmsprop.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_105 (Dense)            (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 4s 4ms/step - loss: 0.1347 - val_loss: 0.3605
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1315 - val_loss: 0.3544
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1309 - val_loss: 0.3505
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1307 - val_loss: 0.3478
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.130

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_rmsprop, best_loss_rmsprop = plot_loss(loss_rmsprop, learning_rate, 'RMSprop learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_rmsprop))
print('Best loss:', str(best_loss_rmsprop))



Best learning rate: 0.001
Best loss: 0.013501342498584763


[链接文字](https://)### Select best optimizer with best learning rate to fine tune other paramters

In [0]:
optimizer = ['Adam','RMSprop','SGD']
best_loss = np.min([best_loss_adam,best_loss_rmsprop,best_loss_sgd])
best_optimizer = optimizer[np.argmin([best_loss_adam,best_loss_rmsprop,best_loss_sgd])]
print('Best optimizer:', best_optimizer)
#  best learning rate 0.0001


Best optimizer: Adam


#### Adam Tuning-parameters

In [0]:
# Adam Get the best number of neurons
time_1 = time()

loss = []
n_neurons_options = [128,256,512]   
for n_neurons in n_neurons_options:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, n_neurons, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_116 (Dense)            (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 5s 7ms/step - loss: 0.1177 - val_loss: 0.3013
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1530 - val_loss: 0.3188
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1447 - val_loss: 0.3255
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1399 - val_loss: 0.3218
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.140

In [0]:
configure_plotly_browser_state()

best_number_of_neurons_Adam, best_loss_Adam = plot_loss(loss,n_neurons_options, 'Adam number of neurons', 'n_neurons')
print('Best_number_of_neurons_Adam:', str(best_number_of_neurons_Adam))
print('Best_loss_Adam:', str(best_loss_Adam))

Best_number_of_neurons_Adam: 128
Best_loss_Adam: 0.013106173528665369


In [0]:
# Get the best number of layers
time_1 = time()

loss = []
n_layers_options = [1,2,3]   
for n_layers in n_layers_options:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, n_layers, 16, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_119 (Dense)            (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 6s 7ms/step - loss: 0.1137 - val_loss: 0.2896
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1549 - val_loss: 0.3112
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1451 - val_loss: 0.3202
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1393 - val_loss: 0.3162
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.139

In [0]:
configure_plotly_browser_state()

best_number_of_layers_Adam, best_loss_Adam = plot_loss(loss,n_layers_options, 'Adam number of layers', 'n_layers')
print('Best_number_of_layers_Adam:', str(best_number_of_layers_Adam))
print('Best_loss_Adam:', str(best_loss_Adam))

Best_number_of_layers_Adam: 1
Best_loss_Adam: 0.013465754564974335


In [0]:
# Get the best number of batch_size
time_1 = time()

loss = []
n_batch_size = [16,1,32,64]   
for batch_size in n_batch_size:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_124 (Dense)            (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 7s 8ms/step - loss: 0.1148 - val_loss: 0.2822
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1529 - val_loss: 0.3028
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1450 - val_loss: 0.3178
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1361 - val_loss: 0.3116
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.136

In [0]:
configure_plotly_browser_state()

best_number_of_batch_size_Adam, best_loss_Adam = plot_loss(loss,n_batch_size, 'Adam batch_size', 'batch_size')
print('Best_number_of_batch_size_Adam:', str(best_number_of_batch_size_Adam))
print('Best_loss_Adam:', str(best_loss_Adam))

Best_number_of_batch_size_Adam: 1
Best_loss_Adam: 0.013254238630307245


In [0]:
# Get the best rate of dropout
time_1 = time()

loss = []
n_dropout_rates = [0,0.25,0.5,0.75]   
for dropout_rate in n_dropout_rates:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 1, 100, dropout_rate)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
761/819 [==========================>...] - ETA: 1s - loss: 0.0139

KeyboardInterrupt: ignored

In [0]:
configure_plotly_browser_state()

best_dropout_rate_Adam, best_loss_Adam = plot_loss(loss,n_dropout_rates, 'Adam dropout_rate', 'dropout_rate')
print('Best_dropout_rate_Adam:', str(best_dropout_rate_Adam))
print('Best_loss_Adam:', str(best_loss_Adam))

Best dropout_rate: 0.25
Best loss: 0.013308201548080522


### GRU

#### Adam

In [0]:
time_1 = time()

# Adam
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]   
loss_adam = []
for lr in learning_rate:
  optimizer = Adam(lr=lr, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_adam.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 128)               49920     
_________________________________________________________________
dense_131 (Dense)            (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 7s 9ms/step - loss: 0.1373 - val_loss: 0.3618
Epoch 2/100
819/819 [==============================] - 1s 969us/step - loss: 0.1326 - val_loss: 0.3545
Epoch 3/100
819/819 [==============================] - 1s 950us/step - loss: 0.1321 - val_loss: 0.3501
Epoch 4/100
819/819 [==============================] - 1s 964us/step - loss: 0.1312 - val_loss: 0.3459
Epoch 5/100
819/819 [==============================] - 1s 959us/step - los

In [0]:
configure_plotly_browser_state()
best_learning_rate_adam, best_loss_adam = plot_loss(loss_adam, learning_rate, 'Adam learning rates for GRU', 'lr')
print('Best learning rate:', str(best_learning_rate_adam))
print('Best loss:', str(best_loss_adam))

Best learning rate: 0.0005
Best loss: 0.013488899251180035


#### SGD

In [0]:
time_1 = time()

# SGD
learning_rate = [1e-4,5e-4,1e-3,5e-3,1e-2]    # MIGHT NEED TO ADJUST
loss_sgd = []
for lr in learning_rate:
  optimizer = SGD(lr=lr, momentum=0.9, decay=1e-6, nesterov=True)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_sgd.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, 128)               49920     
_________________________________________________________________
dense_136 (Dense)            (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 8s 9ms/step - loss: 0.1086 - val_loss: 0.2596
Epoch 2/100
819/819 [==============================] - 1s 916us/step - loss: 0.1468 - val_loss: 0.2803
Epoch 3/100
819/819 [==============================] - 1s 942us/step - loss: 0.1357 - val_loss: 0.2818
Epoch 4/100
819/819 [==============================] - 1s 937us/step - loss: 0.1335 - val_loss: 0.2786
Epoch 5/100
819/819 [==============================] - 1s 905us/step - los

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_sgd, best_loss_sgd = plot_loss(loss_sgd, learning_rate, 'SGD learning rates for GRU', 'lr')
print('Best learning rate:', str(best_learning_rate_sgd))
print('Best loss:', str(best_loss_sgd))

Best learning rate: 0.0005
Best loss: 0.015139419605935013


#### RMSprop

In [0]:
time_1 = time()

# RMSprop
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]    # MIGHT NEED TO ADJUST
loss_rmsprop = []
for lr in learning_rate:
  optimizer = RMSprop(lr=lr, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_rmsprop.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_11 (GRU)                 (None, 128)               49920     
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 8s 10ms/step - loss: 0.1330 - val_loss: 0.3582
Epoch 2/100
819/819 [==============================] - 1s 947us/step - loss: 0.1316 - val_loss: 0.3511
Epoch 3/100
819/819 [==============================] - 1s 964us/step - loss: 0.1315 - val_loss: 0.3466
Epoch 4/100
819/819 [==============================] - 1s 937us/step - loss: 0.1307 - val_loss: 0.3423
Epoch 5/100
819/819 [==============================] - 1s 955us/step - lo

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_rmsprop, best_loss_rmsprop = plot_loss(loss_rmsprop, learning_rate, 'RMSprop learning rates for GRU', 'lr')
print('Best learning rate:', str(best_learning_rate_rmsprop))
print('Best loss:', str(best_loss_rmsprop))

Best learning rate: 5e-05
Best loss: 0.01309153442731598


#### RMSprop Tuning-parameters

In [0]:
# RMSprop Get the best number of neurons
time_1 = time()

loss = []
n_neurons_options = [128,256,512]   
for n_neurons in n_neurons_options:
  optimizer =RMSprop(lr=5e-05, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, n_neurons, 1, 16, 100, 0)
  loss.append(history.history['val_loss'])

time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_17 (GRU)                 (None, 128)               49920     
_________________________________________________________________
dense_147 (Dense)            (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 9s 11ms/step - loss: 0.1049 - val_loss: 0.2690
Epoch 2/100
819/819 [==============================] - 1s 961us/step - loss: 0.1222 - val_loss: 0.2657
Epoch 3/100
819/819 [==============================] - 1s 932us/step - loss: 0.1211 - val_loss: 0.2623
Epoch 4/100
819/819 [==============================] - 1s 959us/step - loss: 0.1199 - val_loss: 0.2587
Epoch 5/100
819/819 [==============================] - 1s 940us/step - lo

In [0]:
configure_plotly_browser_state()

best_number_of_neurons_RMSprop, best_loss_RMSprop = plot_loss(loss,n_neurons_options, 'RMSprop number of neurons', 'n_neurons')
print('Best_number_of_neurons_RMSprop:', str(best_number_of_neurons_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_number_of_neurons_RMSprop: 128
Best_loss_RMSprop: 0.012754747152819738


In [0]:
# Get the best number of layers
time_1 = time()

loss = []
n_layers_options = [1,2,3]   
for n_layers in n_layers_options:
  optimizer =RMSprop(lr=5e-05, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, n_layers, 16, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_20 (GRU)                 (None, 128)               49920     
_________________________________________________________________
dense_150 (Dense)            (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 10s 12ms/step - loss: 0.1158 - val_loss: 0.3115
Epoch 2/100
819/819 [==============================] - 1s 961us/step - loss: 0.1343 - val_loss: 0.3053
Epoch 3/100
819/819 [==============================] - 1s 968us/step - loss: 0.1348 - val_loss: 0.3026
Epoch 4/100
819/819 [==============================] - 1s 946us/step - loss: 0.1337 - val_loss: 0.2995
Epoch 5/100
819/819 [==============================] - 1s 965us/step - l

In [0]:
configure_plotly_browser_state()

best_number_of_layers_RMSprop, best_loss_RMSprop = plot_loss(loss,n_layers_options, 'RMSprop number of neurons', 'n_layers')
print('Best_number_of_layers_RMSprop:', str(best_number_of_layers_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_number_of_layers_RMSprop: 1
Best_loss_RMSprop: 0.01399981707416393


In [0]:
# Get the best number of batch_size
time_1 = time()

loss = []
n_batch_size = [16,1,32,64]   
for batch_size in n_batch_size:
  optimizer =RMSprop(lr=5e-05, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, batch_size, 100, 0)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 128)               49920     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 2s 2ms/step - loss: 0.1041 - val_loss: 0.2737
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1265 - val_loss: 0.2709
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1252 - val_loss: 0.2677
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1238 - val_loss: 0.2637
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.123

In [0]:
configure_plotly_browser_state()

best_batch_size_RMSprop, best_loss_RMSprop = plot_loss(loss,n_batch_size, 'RMSprop batch_size', 'batch_size')
print('Best_batch_size_RMSprop:', str(best_batch_size_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_batch_size_RMSprop: 16
Best_loss_RMSprop: 0.012555881198700313


In [0]:
# Get the best rate of dropout
time_1 = time()

loss = []
n_drop_rates = [0,0.25,0.5,0.75]   
for drop_rate in n_drop_rates:
  optimizer =RMSprop(lr=0.001, decay=1e-6)
  history, model = GRU_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, drop_rate)
  loss.append(history.history['val_loss'])
time_2 = time()
print('Training this model takes '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 128)               49920     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 50,049
Trainable params: 50,049
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 2s 3ms/step - loss: 0.0349 - val_loss: 0.0308
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.0676 - val_loss: 0.0342
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.0584 - val_loss: 0.0334
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.0512 - val_loss: 0.0352
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.047

In [0]:
configure_plotly_browser_state()

best_dropout_rate_RMSprop, best_loss_RMSprop = plot_loss(loss,n_drop_rates, 'RMSprop batch_size', 'drop_rate')
print('Best_dropout_rate_RMSprop:', str(best_dropout_rate_RMSprop))
print('Best_loss_RMSprop:', str(best_loss_RMSprop))

Best_dropout_rate_RMSprop: 0.5
Best_loss_RMSprop: 0.012086403924603384


## **FINAL TEST DATA VISUALIZATION**

In [0]:
# Inverse Y_pred
y_pred = model.predict(MLP_Y_train)
y_true = scaler2.inverse_transform(Y_test)
y_hat = scaler2.inverse_transform(y_pred)


In [0]:
configure_plotly_browser_state()

trace1 = go.Scatter(
    x = np.arange(0, len(y_hat), 1),
    y = y_hat.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(y_true), 1),
    y = y_true.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

rmse = np.mean( np.sqrt((y_true.reshape(-1,) - y_hat.reshape(-1,))**2) )
print(np.sqrt(mean_squared_error(y_true.reshape(-1,), y_hat.reshape(-1,))))


578.3576640016041
